In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import time

In [2]:
JAR_HOME = "/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir"

JAR_FILE_NAME = ["commons-pool2-2.11.1.jar", "kafka-clients-3.1.0.jar", 
                 "spark-sql-kafka-0-10_2.12-3.2.1.jar", "spark-token-provider-kafka-0-10_2.12-3.2.1.jar"]

In [3]:
JAR_PATH = ""

for file_name in JAR_FILE_NAME:
    JAR_PATH += f'{JAR_HOME}/{file_name},'

JAR_PATH = JAR_PATH[:-1]

In [4]:
# JAR_PATH = "/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/spark-sql-kafka-0-10_2.12-3.2.1.jar,/home/khanhpluto/khanhtv/kafka-clients-3.1.0.jar"

In [5]:
JAR_PATH

'/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/commons-pool2-2.11.1.jar,/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/kafka-clients-3.1.0.jar,/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/spark-sql-kafka-0-10_2.12-3.2.1.jar,/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/spark-token-provider-kafka-0-10_2.12-3.2.1.jar'

In [6]:
kafka_topic_name = "topic1"
kafka_bootstrap_servers = 'localhost:9092'

In [7]:
spark = SparkSession.builder.appName("Spark kafka") \
    .config("spark.jars", JAR_PATH) \
    .config("spark.executor.extraClassPath", JAR_PATH) \
    .config("spark.executor.extraLibrary", JAR_PATH) \
    .config("spark.driver.extraClassPath", JAR_PATH) \
    .config("spark.driver.userClassPathFirst", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

/home/khanhpluto/khanhtv/test_spark/zxc/venv/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [8]:
flower_df = spark \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
            .option("subscribe", kafka_topic_name) \
            .option("startingOffsets", "latest") \
            .load()

flower_df1 = flower_df.selectExpr("CAST(value AS STRING)", "timestamp")


flower_schema_string = "sepal_length DOUBLE,sepal_width DOUBLE,petal_length DOUBLE,petal_width DOUBLE,species STRING"

In [9]:
flower_df2 = flower_df1 \
        .select(from_csv(col("value"), flower_schema_string) \
                .alias("flower"), "timestamp")


flower_df3 = flower_df2.select("flower.*", "timestamp")

    
flower_df3.createOrReplaceTempView("flower_find");
song_find_text = spark.sql("SELECT * FROM flower_find")
flower_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("testedTable") \
        .start()

flower_agg_write_stream.awaitTermination(1)

False

In [30]:
df = spark.sql("SELECT * FROM testedTable")
df.show()

+------------+-----------+------------+-----------+-------+--------------------+
|sepal_length|sepal_width|petal_length|petal_width|species|           timestamp|
+------------+-----------+------------+-----------+-------+--------------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|2022-04-14 16:02:...|
|         4.9|        3.0|         1.4|        0.2| Setosa|2022-04-14 16:02:...|
|         4.7|        3.2|         1.3|        0.2| Setosa|2022-04-14 16:02:...|
|         4.6|        3.1|         1.5|        0.2| Setosa|2022-04-14 16:02:...|
|         5.0|        3.6|         1.4|        0.2| Setosa|2022-04-14 16:02:...|
|         5.4|        3.9|         1.7|        0.4| Setosa|2022-04-14 16:02:...|
|         4.6|        3.4|         1.4|        0.3| Setosa|2022-04-14 16:02:...|
|         5.0|        3.4|         1.5|        0.2| Setosa|2022-04-14 16:02:...|
|         4.4|        2.9|         1.4|        0.2| Setosa|2022-04-14 16:02:...|
|         4.9|        3.1|  

In [31]:
df.count()

107